In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

In [2]:
# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [3]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"]) 

In [4]:
school_data_complete.head ()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
#Calculate the total number of schools
schools_total = len(school_data_complete["school_name"].unique())

#Calculate the total number of students
students_total = len(school_data_complete["student_name"])

#Calculate the total budget
budget_per_school = school_data_complete["budget"].unique()
budget_total = sum(budget_per_school)
budget_total_num = budget_total.astype(float)

#Calculate the average math score
math_average = school_data_complete["math_score"].mean()

#Calculate the average reading score
reading_average = school_data_complete["reading_score"].mean()

#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
pass_overall = (math_average+reading_average)/2

#Calculate the percentage of students with a passing math score (70 or greater)
pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
pass_math_percent = len(pass_math)/students_total*100

#Calculate the percentage of students with a passing reading score (70 or greater)
pass_read = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
pass_read_percent = len(pass_read)/students_total*100

#Create a dataframe to hold the above results
data_summary1 = pd.DataFrame({"Total Schools":[schools_total], "Total Students":[students_total], "Total Budget":[budget_total_num], 
    "Average Math Score":[math_average], "Average Reading Score":[reading_average], "% Passing Math":[pass_math_percent], 
    "% Passing Reading":[pass_read_percent], "% Overall Passing Rate":[pass_overall]
    }
)
#Optional: give the displayed data cleaner formatting
data_summary1["Total Budget"] = data_summary1["Total Budget"].map('${:,.2f}'.format)

data_summary1

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


In [6]:
# Create an overview table that summarizes key metrics about each school, including:
    # School Name
    # School Type
    # Total Students
    # Total School Budget
    # Per Student Budget
    # Average Math Score
    # Average Reading Score
    # % Passing Math
    # % Passing Reading
    # Overall Passing Rate (Average of the above two)
# Create a dataframe to hold the above results

# _________________________________________________________________________________________

# FORMAT ALL DATAFRAMES THAT WILL BE CALLED UPON
# _________________________________________________________________________________________
school_data_trim = school_data.drop(["School ID"], axis=1) 
school_data_index = school_data_trim.set_index("school_name")
reduced_school_data_complete = school_data_complete.drop(["student_name", "gender", "School ID"], axis=1)
reduced_school_data_complete_index = reduced_school_data_complete.set_index("school_name")
# _________________________________________________________________________________________

# CREATE DATA SUMMARY DF AND CALCULATE BUDGET PER STUDENT
# _________________________________________________________________________________________

data_summary2 = school_data_index
data_summary2["Per Student Budget"] = data_summary2["budget"]/data_summary2["size"]
# _________________________________________________________________________________________

# FIND AVERAGE MATH & READING SCORES PER SCHOOL
# _________________________________________________________________________________________

# use .mean on math_score and reading_score while .groupby school_name to find average scores
math_avg_per_school = reduced_school_data_complete.groupby(["school_name"],as_index=True)["math_score"].mean()
reading_avg_per_school = reduced_school_data_complete.groupby(["school_name"],as_index=True)["reading_score"].mean()

# add columns with average math & reading scores per school to data_summary2
data_summary2["Average Math Score"] = math_avg_per_school
data_summary2["Average Reading Score"] = reading_avg_per_school
# _________________________________________________________________________________________

# FIND PERCENT WHO PASSED MATH PER SCHOOL
# _________________________________________________________________________________________

# filter out students who failed math
filtered_student_data_math = reduced_school_data_complete_index.loc[(reduced_school_data_complete_index["math_score"]>=70)]

# group filtered_student_data_math by school name
grouped_filtered_student_data_math = filtered_student_data_math.groupby(["school_name"])

# use .count to find number of students who passed math for each school
num_stuperschool_pass_math = grouped_filtered_student_data_math["math_score"].count()

# calculate numbe_stuperschool_pass_math/total size of school*100 to find percent passed
data_summary2["% Passing Math"] = num_stuperschool_pass_math/data_summary2["size"]*100
# _________________________________________________________________________________________

# FIND PERCENT WHO PASSED READING PER SCHOOL
# _________________________________________________________________________________________

# filter out students who failed reading
filtered_student_data_reading = reduced_school_data_complete_index.loc[(reduced_school_data_complete_index["reading_score"]>=70)]

# group filtered_student_data_reading by school name
grouped_filtered_student_data_reading = filtered_student_data_reading.groupby(["school_name"])

# use .count to find number of students who passed math for each school
num_stuperschool_pass_reading = grouped_filtered_student_data_reading["reading_score"].count()

# calculate num_stuperschool_pass_reading/total size of school*100 to find percent passed
data_summary2["% Passing Reading"] = num_stuperschool_pass_reading/data_summary2["size"]*100
# _________________________________________________________________________________________

# FIND OVERALL PASSED PER SCHOOL
# _________________________________________________________________________________________

# divide sum of % passing math & % passing reading over total students * 100
percent_stuperschool_pass_overall = data_summary2["% Passing Math"]+data_summary2["% Passing Reading"]
data_summary2["% Overall Passing Rate"] = percent_stuperschool_pass_overall/2
# _________________________________________________________________________________________

# RENAME/REMOVE COLUMNS & FORMAT DOLLARS
# _________________________________________________________________________________________

data_summary2_cleaned = data_summary2.rename(columns={"type":"School Type", "size":"Total Students", "budget":"Total School Budget"})
data_summary2_cleaned["Total School Budget"] = data_summary2_cleaned["Total School Budget"].map('${:,.2f}'.format)
data_summary2_cleaned["Per Student Budget"] = data_summary2_cleaned["Per Student Budget"].map('${:,.2f}'.format)
# _________________________________________________________________________________________

# RENDER DATA SUMMARY
# _________________________________________________________________________________________

data_summary2_cleaned.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668


In [7]:
#Sort and display the top five schools in overall passing rate
data_summary2_cleaned.sort_values("% Overall Passing Rate", ascending=False).head(n=5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [8]:
#Sort and display the five worst-performing schools
data_summary2_cleaned.sort_values("% Overall Passing Rate", ascending=True).head(n=5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [9]:
#Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
#Create a pandas series for each grade. Hint: use a conditional statement.
#Group each series by school
grade_9m = student_data.loc[student_data["grade"]=="9th"].groupby(["school_name"], as_index=False)
mean_math_9 = grade_9m["math_score"].mean()
grade_10m = student_data.loc[student_data["grade"]=="10th"].groupby(["school_name"], as_index=False)
mean_math_10 = grade_10m["math_score"].mean()
grade_11m = student_data.loc[student_data["grade"]=="11th"].groupby(["school_name"], as_index=False)
mean_math_11 = grade_11m["math_score"].mean()
grade_12m = student_data.loc[student_data["grade"]=="12th"].groupby(["school_name"], as_index=False)
mean_math_12 = grade_12m["math_score"].mean()

merge_m9th10th = pd.merge(mean_math_9, mean_math_10, on="school_name")
merge_m9th10th = merge_m9th10th.rename(columns={"math_score_x": "9th", "math_score_y": "10th"})

merge_m11th12th = pd.merge(mean_math_11, mean_math_12, on="school_name")
merge_m11th12th = merge_m11th12th.rename(columns={"math_score_x": "11th", "math_score_y": "12th"})

avg_math_per_grade = pd.merge(merge_m9th10th, merge_m11th12th, on="school_name")
avg_math_per_grade = avg_math_per_grade.set_index("school_name")

#Optional: give the displayed data cleaner formatting

del avg_math_per_grade.index.name
avg_math_per_grade.head()

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


In [10]:
#Perform the same operations as above for reading scores
grade_9r = student_data.loc[student_data["grade"]=="9th"].groupby(["school_name"], as_index=False)
mean_reading_9 = grade_9r["reading_score"].mean()
grade_10r = student_data.loc[student_data["grade"]=="10th"].groupby(["school_name"], as_index=False)
mean_reading_10 = grade_10r["reading_score"].mean()
grade_11r = student_data.loc[student_data["grade"]=="11th"].groupby(["school_name"], as_index=False)
mean_reading_11 = grade_11r["reading_score"].mean()
grade_12r = student_data.loc[student_data["grade"]=="12th"].groupby(["school_name"], as_index=False)
mean_reading_12 = grade_12r["reading_score"].mean()

merge_r9th10th = pd.merge(mean_reading_9, mean_reading_10, on="school_name")
merge_r9th10th = merge_r9th10th.rename(columns={"reading_score_x": "9th", "reading_score_y": "10th"})

merge_r11th12th = pd.merge(mean_reading_11, mean_reading_12, on="school_name")
merge_r11th12th = merge_r11th12th.rename(columns={"reading_score_x": "11th", "reading_score_y": "12th"})

avg_reading_per_grade = pd.merge(merge_r9th10th, merge_r11th12th, on="school_name")
avg_reading_per_grade = avg_reading_per_grade.set_index("school_name")

del avg_reading_per_grade.index.name
avg_reading_per_grade.head()

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [11]:
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
#Average Math Score
#Average Reading Score
#% Passing Math
#% Passing Reading
#Overall Passing Rate (Average of the above two)
# Sample bins. Feel free to create your own bins.


spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#use Per Student Budget from data_summary2 NOT data_summary2_cleaned (formatted) to create bins data series 
data_summary2["Spending Ranges (Per Student)"] = pd.cut(data_summary2["Per Student Budget"], spending_bins, labels=group_names)

# create new object based on Spending Range (Per Student)
data_summary_spend = data_summary2.groupby("Spending Ranges (Per Student)")
# create data frame based on .mean amount within every bin range for desired variables
data_summary_spend[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [12]:
#Perform the same operations as above, based on school size. 
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

data_summary2["School Size"] = pd.cut(data_summary2["size"], size_bins, labels=group_names)

data_summary_spend = data_summary2.groupby("School Size")

data_summary_spend[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [13]:
#Perform the same operations as above, based on school type.

data_summary_spend = data_summary2.groupby("type")

data_summary_spend[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
